In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
dataset= pd.read_csv('/home/DEADPOOL/Downloads/ag_news.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119999 entries, 0 to 119998
Data columns (total 3 columns):
Unnamed: 0    119999 non-null int64
label         119999 non-null int64
headline      119999 non-null object
dtypes: int64(2), object(1)
memory usage: 2.7+ MB


In [4]:
dataset.head()

,Unnamed: 0,label,headline
0,0,3,Carlyle Looks Toward Commercial Aerospace (Reu...
1,1,3,Oil and Economy Cloud Stocks' Outlook (Reuters)
2,2,3,Iraq Halts Oil Exports from Main Southern Pipe...
3,3,3,"Oil prices soar to all-time record, posing new..."
4,4,3,"Stocks End Up, But Near Year Lows (Reuters)"


In [5]:
headlines=dataset['headline'].values
labels=dataset['label'].values

In [6]:
train_headlines=headlines[:75000]
train_labels=labels[:75000]

test_headlines=headlines[75000:]
test_labels=labels[75000:]

train_headlines.shape,test_headlines.shape

((75000,), (44999,))

In [14]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import re

In [15]:
def strip_html_tags(text):
    soup = BeautifulSoup(text,'html.parser')
    [s.extract() for s in soup(['iframe','script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+','\n',stripped_text)
    return stripped_text

In [16]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8','ignore')
    return text

In [17]:
def expand_contractions(text):
    return contractions.fix(text)

In [18]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

In [19]:
def pre_process_document(document):
    # strip HTML
    document = strip_html_tags(document)
    # lower case
    document = document.lower()
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    # remove accented characters
    document = remove_accented_chars(document)
    # expand contractions    
    document = expand_contractions(document)  
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)  
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document

pre_process_corpus = np.vectorize(pre_process_document)

In [21]:
train_headlines = pre_process_corpus(train_headlines)
#val_headlines = pre_process_corpus(val_headlines)
test_headlines = pre_process_corpus(test_headlines)

AttributeError: module 'contractions' has no attribute 'fix'